### Stockfish vs Stockfish

In [ ]:
import os

import autorootcwd
import chess.engine
import chess.pgn
from rich.console import Console
from rich.table import Table

from src.utils.chess_utils import ChessBoard

console = Console()


def stockfish_vs_stockfish(stockfish_path: str, output_data: str, cpu_nums: int, time_limit: float, num_games: int):
    """Play a game between two Stockfish instances.

    Args:
        stockfish_path (str): Path to the Stockfish executable.
        cpu_nums (int, optional): Number of CPUs to be used by Stockfish. Defaults to 4.
        time_limit (float, optional): Time in seconds each engine has to make a move. Defaults to 5.0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    os.makedirs(output_data, exist_ok=True)

    stockfish_total_time = 0
    for i in range(num_games):
        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

        # Configure Stockfish to use more threads for increased speed
        engine.configure({"Threads": cpu_nums})

        game = chess.pgn.Game()
        node = game

        while not board.is_game_over():
            result = engine.play(board, chess.engine.Limit(time=time_limit))
            stockfish_total_time = result.info["time"]
            move = result.move
            board.push(move)
            ChessBoard().show(board, False)

            node = node.add_variation(move)

        engine.quit()

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Stockfish"
        game.headers["Black"] = "Stockfish"

        # Save to a PGN file
        with open(f"{output_data}/stockfish_vs_stockfish_{i}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)

    stockfish_avg_time = stockfish_total_time / num_games

    table = Table(title="Stockfish vs Stockfish", show_header=True, header_style="bold magenta")
    table.add_column("Games", style="dim", width=12)
    table.add_column("Avg. Time", style="dim", width=12)
    table.add_column("Total Time", style="dim", width=12)
    table.add_row(str(num_games), str(stockfish_avg_time), str(stockfish_total_time))
    console.print(table)

    return game


if __name__ == "__main__":
    stockfish_path = "engine/stockfish/stockfish-ubuntu-x86-64-avx2"
    output_data = "data/trained_data"
    cpu_nums = 4
    time_limit = 5.0
    num_games = 1
    stockfish_vs_stockfish(stockfish_path, output_data, cpu_nums, time_limit, num_games)

### lc0 vs lc0

In [ ]:
import os

import autorootcwd
import chess.engine
import chess.pgn
from rich.console import Console
from rich.table import Table

from src.utils.chess_utils import ChessBoard

console = Console()


def lc0_vs_lc0(lc0_path: str, output_data: str, weights_file: str, num_games: int):
    """Play a game between two Lc0 instances.

    Args:
        lc0_path (str): Path to the Lc0 executable.
        weights_file (str): Path to the weights file for Lc0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    os.makedirs(output_data, exist_ok=True)
    lc0_total_time = 0
    for i in range(num_games):
        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(lc0_path)

        # Configure Lc0 with the weights file
        engine.configure({"WeightsFile": weights_file})

        game = chess.pgn.Game()
        node = game

        while not board.is_game_over():
            result = engine.play(board, chess.engine.Limit(time=5.0))
            lc0_total_time += result.info["time"]
            move = result.move
            board.push(move)
            ChessBoard().show(board, False)

            node = node.add_variation(move)

        engine.quit()

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Lc0"
        game.headers["Black"] = "Lc0"

        # Save to a PGN file
        with open(f"{output_data}/lc0_vs_lc0_{i}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)

    lc0_avg_time = lc0_total_time / num_games

    table = Table(title="Lc0 vs Lc0", show_header=True, header_style="bold magenta")
    table.add_column("Games")
    table.add_column("Avg. Time")
    table.add_column("Total Time")
    table.add_row(f"{num_games}", f"{lc0_avg_time:.2f}s", f"{lc0_total_time:.2f}s")
    console.print(table)

    return game


if __name__ == "__main__":
    lc0_path = "engine/Lc0/lc0.exe"
    output_data = "data/trained_data"
    weights_file = "engine/Lc0/client-cache/ac8c69b197421a39d5bd407ff389138d36539b7801b6f7d068d90b16f43e10c0"
    num_games = 1
    lc0_vs_lc0(lc0_path, output_data, weights_file, num_games)

### lc0 vs Stockfish

In [ ]:
import os

import autorootcwd
import chess.engine
import chess.pgn
from rich.console import Console
from rich.table import Table

from src.utils.chess_utils import ChessBoard

console = Console()


def lc0_vs_stockfish(lc0_path: str, output_data: str, weights_file: str, stockfish_path: str, num_games: int):
    os.makedirs(output_data, exist_ok=True)

    lc0_wins = 0
    stockfish_wins = 0
    draws = 0
    lc0_total_time = 0
    stockfish_total_time = 0

    for i in range(num_games):
        board = chess.Board()
        lc0_engine = chess.engine.SimpleEngine.popen_uci(lc0_path)
        stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

        # Configure Lc0 with the weights file
        lc0_engine.configure({"WeightsFile": weights_file})

        game = chess.pgn.Game()
        node = game

        while not board.is_game_over():
            if board.turn == chess.WHITE:
                result = lc0_engine.play(board, chess.engine.Limit(time=5.0))
                lc0_total_time += result.info["time"]
            else:
                result = stockfish_engine.play(board, chess.engine.Limit(time=5.0))
                stockfish_total_time += result.info["time"]
            move = result.move
            board.push(move)
            ChessBoard().show(board, False)

            node = node.add_variation(move)

        lc0_engine.quit()
        stockfish_engine.quit()

        # Update win counters
        if board.result() == '1-0':
            lc0_wins += 1
        elif board.result() == '0-1':
            stockfish_wins += 1
        else:
            draws += 1

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Lc0"
        game.headers["Black"] = "Stockfish"

        # Save to a PGN file
        with open(f"{output_data}/lc0_vs_stockfish_{i}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)

    lc0_winrate = (lc0_wins / num_games) * 100
    stockfish_winrate = (stockfish_wins / num_games) * 100
    drawrate = (draws / num_games) * 100
    lc0_avg_time = lc0_total_time / num_games
    stockfish_avg_time = stockfish_total_time / num_games

    table = Table(title="Lc0 vs Stockfish Results")
    table.add_column("Lc0 Winrate")
    table.add_column("Stockfish Winrate")
    table.add_column("Draw Rate")
    table.add_column("Total Games")
    table.add_row(f"{lc0_winrate}% ({lc0_winrate})", f"{stockfish_winrate}% ({stockfish_wins})", f"{drawrate}", f"{num_games}")
    console.print(table)

    time_table = Table(title="Lc0 vs Stockfish Average Time")
    time_table.add_column("Lc0 Time")
    time_table.add_column("Stockfish Time")
    time_table.add_row(f"{lc0_avg_time}", f"{stockfish_avg_time}")
    console.print(time_table)

    return game


if __name__ == "__main__":
    lc0_path = "engine/Lc0/lc0.exe"
    output_data = "data/trained_data"
    weights_file = "engine/Lc0/client-cache/ac8c69b197421a39d5bd407ff389138d36539b7801b6f7d068d90b16f43e10c0"
    stockfish_path = "engine/stockfish/stockfish-windows-x86-64-avx2.exe"
    num_games = 600
    lc0_vs_stockfish(lc0_path, output_data, weights_file, stockfish_path, num_games)